In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
base_path = "C:\\"
lending_fil = base_path + "loan.zip"
pd_lend_df = pd.read_csv(lending_fil)
pd_lend_df.info()

In [ ]:
pd_lend_df.dropna(axis = 1, how = 'all' ,inplace = True)
pd_lend_df.info()
#pd_numeric = pd_lend_df.describe()
#pd_numeric.info()

In [ ]:
pd_unique=pd_lend_df.loc[:,pd_lend_df.nunique()==1].copy()
pd_unique.describe(include = 'all')


In [ ]:
pd_unique = pd.DataFrame()
pd_lend_df=pd_lend_df.loc[:,pd_lend_df.nunique()>1]

In [ ]:
pd_lend_df.info()

In [ ]:
pd_lend_df.describe().to_csv(base_path + "lend_num.csv")
#print(len(pd_lend_df['member_id'].unique()))
#print(len(pd_lend_df['id'].unique()))

In [ ]:
pd_lend_df.describe(include ='object')

In [ ]:
print(len(pd_lend_df[pd_lend_df['term'].str.contains("months") == True]))

In [ ]:
pd_lend_df['term'] = pd_lend_df['term'].str.replace("months" ,"")
pd_lend_df.describe(include ='object')

In [ ]:
print(len(pd_lend_df[pd_lend_df['int_rate'].str.contains("%") == True]))

In [ ]:
pd_lend_df['int_rate'] = pd_lend_df['int_rate'].str.replace("%" ,"")
pd_lend_df.describe(include ='object')

In [ ]:
print(len(pd_lend_df[pd_lend_df['revol_util'].str.contains("%") == True]))

In [ ]:
pd_lend_df['revol_util'] = pd_lend_df['revol_util'].str.replace("%" ,"")
pd_lend_df.describe(include ='object')

In [ ]:
print(len(pd_lend_df[pd_lend_df['emp_length'].str.contains("years") == True]))
print(pd_lend_df['emp_length'].unique())

In [ ]:
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("< 1" ,"0")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("+" ,"")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("years" ,"")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("year" ,"")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace(" " ,"")
print(pd_lend_df['emp_length'].unique())
#pd_lend_df.describe(include ='object')



In [ ]:
pd_lend_df['revol_util'] = pd_lend_df['revol_util'].astype(float)
pd_lend_df['int_rate']  = pd_lend_df['int_rate'].astype(float)
print(pd_lend_df['emp_length'].isna().sum())
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].fillna(-1) # replace NaN with -1 , need to see  if this is correct
pd_lend_df['emp_length']  = pd_lend_df['emp_length'].astype(int)
pd_lend_df['term']  = pd_lend_df['term'].astype(int)


In [ ]:
pd_lend_df['zip_code'] = pd_lend_df['zip_code'].str.replace("xx" ,"")
pd_lend_df.describe(include ='object')

In [ ]:
print(pd_lend_df['next_pymnt_d'].unique())
print(pd_lend_df['addr_state'].unique())
print(pd_lend_df['purpose'].unique())  # might be imp
print(pd_lend_df['verification_status'].unique())
print(pd_lend_df['home_ownership'].unique())
print(pd_lend_df['grade'].unique())
print(pd_lend_df['sub_grade'].unique())
print(pd_lend_df['loan_status'].unique())
print(len(pd_lend_df['title'].unique()))

In [ ]:
print(len(pd_lend_df['url'].unique()))
len_url = pd_lend_df['url'].str.len()
len_url.describe()

In [ ]:
len_desc = pd_lend_df['desc'].str.len()
len_desc.describe()

In [ ]:
#from urllib.parse import urlparse
import urllib.parse as urlparse
pd_lend_df['protocol'],pd_lend_df['domain'],pd_lend_df['path'],pd_lend_df['query'],pd_lend_df['fragment'] = zip(*pd_lend_df['url'].map(urlparse.urlsplit))
#pd_lend_df['url']= zip(*pd_lend_df['url'].map(urlparse.urlsplit))[3]
#print(pd_lend_df['protocol'].unique())
#print(pd_lend_df['domain'].unique())
#print(pd_lend_df['path'].unique())
pd_lend_df['query'] = pd_lend_df['query'].str.replace("loan_id=","")
#print(pd_lend_df['query'].unique())
#print(pd_lend_df['fragment'].unique())
pd_lend_df.drop(columns = ['protocol','domain','path','fragment','url'] ,inplace = True)
pd_lend_df.describe(include ='object')

In [ ]:
#rint(len(pd_lend_df['query'].unique()))
#print(pd_lend_df['query'].unique())

In [ ]:
pd_lend_df['desc'].head()


In [ ]:
pd_lend_df['desc'].tail()

In [ ]:
pd_lend_df['last_pymnt_d'].unique()

In [ ]:
pd_lend_df.corr().round(2).to_csv(base_path + "corr_lendingclub.csv")


In [ ]:
pd_lend_df.drop(columns=['desc','funded_amnt_inv','out_prncp_inv','total_pymnt_inv','total_pymnt_inv','total_pymnt',
                        'next_pymnt_d','query','member_id','emp_title', 'title','pub_rec_bankruptcies','out_prncp'], inplace = True)
#desc is long text column, some of this info is in purpose and tile tolumns
#funded_amnt_inv , loan amt and funded amt are highly correlated, dropping one for now
#out_prncp_inv is highly correlated out_prncp; similar case for total_pymnt_inv ,total_pymnt_inv
# total_pymnt = total_rec_prncp + total_rec_int , hence dropping princ and int cols
# next_pymnt_d has lot of blanks and same vals , drop
#query and # id same , drop query , member_id all unique , id will suffice as index , so drop member_id
# emp_title  has lot of unique values and is mix of employer name + title , drop
# title : too many unique vals , drop , instead use purpose
# pub_rec_bankruptcies kind of same as pub_rec -- look at dictionary
#out_prncp applies to current loans, drop
#,'zip_code' -- evaluate dropping of the same

In [ ]:
# convert object to dates
pd_lend_df['issue_d'] = pd.to_datetime(pd_lend_df['issue_d'],format = '%b-%y' )
pd_lend_df['last_pymnt_d'] = pd.to_datetime(pd_lend_df['last_pymnt_d'],format = '%b-%y' )
pd_lend_df['last_credit_pull_d'] = pd.to_datetime(pd_lend_df['last_credit_pull_d'],format = '%b-%y' )
pd_lend_df['earliest_cr_line'] = pd.to_datetime(pd_lend_df['earliest_cr_line'],format = '%b-%y' )
#print(type(pd_lend_df['issue_d']), type(pd_lend_df['last_pymnt_d']))
pd_lend_df['issue_d'].head()
pd_lend_df['last_pymnt_d'].head()
pd_lend_df['last_credit_pull_d'].head()

In [ ]:
fig = px.box(pd_lend_df, x="loan_amnt",y="loan_status")
fig.update_traces(quartilemethod="linear") # or "inclusive", or "linear" by default
fig.show()

In [ ]:
fig = px.histogram(pd_lend_df, x="loan_amnt")
fig.show()

In [ ]:
fig = px.pie(pd_lend_df, names='loan_status')
fig.show()

In [ ]:
# loan_status of current means ongoing loan and is very small no. , dropping such rows
pd_lend_df = pd_lend_df[~(pd_lend_df['loan_status'] == "Current")]
pd_lend_df.to_csv(base_path + "upd1_lendingclub.csv" ,index = False)
pd_lend_df.corr().round(2).to_csv(base_path + "corr_lendingclub.csv")

In [ ]:
fig = px.bar(pd_lend_df, y="annual_inc" , x = "purpose",color = "loan_status")
#fig.update_traces(quartilemethod="linear") # or "inclusive", or "linear" by default
fig.show()